# 1样本与数据说明
## 1.1样本
    此次是参照宫晓莉（2020）文献中样本的选择，对中国在2007年之前上市的所有银行进行研究，主要是为了包括08年次贷危机这个时间段进行研究，14家银行分别为，平安银行(000001.SZ), 浦发银行(600000.SH), 华夏银行(600015.SH), 民生银行(600016.SH), 招商银行(600036.SH), 兴业银行(601166.SH), 交通银行(601328.SH), 工商银行(601398.SH), 建设银行(601939.SH), 中国银行(601988.SH), 中信银行(601998.SH), 宁波银行(002142.SZ), 南京银行(601009.SH), 北京银行(601169.SH)。

    此外关于金融system的定义为所有上市的银行，即会动态包括更新上市的新银行，这样会可以使得对金融system的定义没有偏误，比如农业银行（2010年上市），苏州银行（2019年上市）等其他银行，加上原来研究的14家银行，在最近的金融系统中一共有40家上市银行（即截止2021年8月6号，中国一共有40家上市的银行，这些银行构成了新的金融系统）
    
    样本的时间跨度为2007年至今


## 1.2数据说明

    数据的频率单位为周，即t-1期和t期的时间间隔为一周
    以下所有数据都来源于Wind
    
    银行层面的数据
    1.银行股价的周度涨幅（即每周的收益率）
    2.金融system的周度市值涨跌幅，即所有上市银行的市值总和的变动情况，即为金融系统的收益率
    
    宏观变量的数据（参考欧阳资生，2020，中国管理科学）
    1.股票市场收益率，选择上证综指每周的收益率数据，源于Wind的指数部分的指数行情序列-上证综合指数
    2.股票市场的波动率，选取上证综指周波动率表示，源于Wind的指数部分的制指数行情序列-上证综合指数-周行情-周振幅
    3.短期流动性利差，用6个月期shibor（上海银行间同业拆放利率）和6个月国债利率之差表示，shibor可以从Wind的债券-多维数据-利率走势分析-货币市场-拆借利率中获得，国债的数据也可以从债券-多维数据-利率走势分析-债券市场-到期收益率（中证）-国债收益率曲线中获得，下面的其他债券收益率都可以从债券市场-到期收益率（中证）中获得
    4.利率风险，选取6个月国债利率，用其前后期到期收益率的差表示，即t期和t-1期的差
    5.利率期限结构，用t时刻6个月国债利率与10年国债利率之差，减去t-1时期的6个月的国债利率与10年期国债利率之差表示
    6.违约风险，用t时期10年期国债利率与10年期AA+级信用登记企业债之差，减去t-1时期10年期国债利率与10年期AA+级信用登记企业债之差表示
    7.房地产收益率，选取申万房地产指数每周收盘价表示，数据来源于Wind数据库
    
    CoVaR原文宏观变量的选择（这是对应美国市场的宏观变量）
    1.The weekly equity market return from CRSP.
    2.VIX, which captures the implied volatility in the stock market reported by the Chicago Board Options Exchange
    3.A short term “liquidity spread,” defined as the difference between the three-month repo rate and the three-month bill rate. This liquidity spread measures short-term liquidity risk.
    4.The change in the three-month Treasury bill rate from the Federal Reserve Board’s H.15. 
    5.The change in the slope of the yield curve, measured by the yield spread between the ten-year Treasury rate and the three-month bill rate obtained from the Federal Reserve Board’s H.15 release.
    6.The change in the credit spread between BAA-rated bonds and the Treasury rate (with the same maturity of ten years) from the Federal Reserve Board’s H.15 release.
    7.The weekly real estate sector return in excess of the market return
    
    宏观变量的选取主要是为了求Time-varying的CoVaR，另外CoVaR原文的宏观变量选择主要是美国市场的宏观变量选取，为了转换到中国银行宏观变量的选取，这里我们参考了嵌入网络舆情指数的中国金融机构系统性风险传染效应研究（欧阳资生，2020，中国管理科学），因为目前中文文献有标明宏观变量的选择的暂时只有这一篇。

# 2 数据读取与整理

导入一些必要的包

In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import gmean
import math
from pandas import Series, DataFrame
import time
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from pandas.core.frame import DataFrame
from scipy.integrate import odeint
from scipy.optimize import root,fsolve
import scipy.stats as stats

## 2.1 银行总市值数据
读取银行总市值的数据，即market value $V_E$  
**总市值的单位为元**

In [192]:
#将14家银行的excel名称列成一个列表
bank14=['000001.SZ.xlsx','600000.SH.xlsx','600015.SH.xlsx','600016.SH.xlsx','600036.SH.xlsx','601166.SH.xlsx',\
        '601328.SH.xlsx','601398.SH.xlsx','601939.SH.xlsx','601988.SH.xlsx','601998.SH.xlsx','002142.SZ.xlsx',\
       '601009.SH.xlsx','601169.SH.xlsx']
#记录想要研究的银行的数目
bank_research_len=len(bank14)

bank14_name=[] #用来记录14家银行的名称
bank14_df=[] #用来记录14家银行的df数据
bank14_code=[] #用记录14家对应的Wind的代码
bank14_ret_df=[]
#开始读取14家银行的数据
for i in range(bank_research_len):
    path_name='bank_data/'+bank14[i] #这是读取文件的路径 如果报错的话 查看一下路径对不对
    excel_read=pd.read_excel(path_name)
    #因为Wind导出的数据后面几列有数据来源：Wind的这几行，会产生NaN值，所以删掉
    excel_read=excel_read.dropna(axis=0,how='any') 
    #记录导入的14家银行的名称
    bank14_name.append(excel_read['简称'].iloc[0])
    #记录14家银行对应的Wind的代码
    bank14_code.append(excel_read['代码'].iloc[0])
    bank14_df.append(excel_read) #将该银行数据加入df列表

上面这份数据包含了银行股价的涨跌，涨跌幅和总市值数据，因为我们也只需要用到‘总市值’的数据，所以把这个提取出来
并且将总市值单位转换为‘万元’，与后续统一

In [193]:
bank14_market_value=[]
for i in range(bank_research_len):
    #市值单位转换为万元
    bank14_df[i]['总市值(元)']=bank14_df[i]['总市值(元)']/10000
    df_temp=bank14_df[i][['日期','总市值(元)']]
    df_temp=bank14_df[i].rename(columns={'总市值(元)':'market_value'})
    bank14_market_value.append(df_temp)

## 2.2 宏观变量数据
下一步提取宏观变量的数据，这里需要注意一下，如果后续要改程序的话，由于不同宏观变量的导出方式不同，可能要注意调整一下，而且有时不同的Wind导出来的格式可能也会存在差异，比如有些excel出来的前面2年都是变量名字（列名），有些是只有前面1行是列名

In [146]:
#宏观变量的文件名
marco_excel_name=['国债与企业债利率-20210806.xlsx','上证收益率和波动率.xlsx','SHIBOR0806.xlsx','房地产行情数据.xlsx'\
                  ,'国债1年收益率.xlsx']

marco_excel_len=len(marco_excel_name)

marco_df=[] #用来记录14家银行的df数据
#开始读取14家银行的数据
for i in range(marco_excel_len):
    path_name='marco_data/'+marco_excel_name[i] #这是读取文件的路径 如果报错的话 查看一下路径对不对
    excel_read=pd.read_excel(path_name)
    excel_read=excel_read.drop(axis=0, index=0, columns=None, inplace=False)
    #因为Wind导出的数据后面几列有数据来源：Wind的这几行，会产生NaN值，所以删掉
    excel_read=excel_read.dropna(axis=0,how='any') 
    marco_df.append(excel_read) #将该银行数据加入df列表

#因为宏观变量的数据不像银行数据批量导出的那么整齐，不同excel中变量数目也不同，所以这几个宏观变量要单独处理，没有for循环那么方便了
#后续要改程序的话，由于不同宏观变量的导出方式不同，可能要注意调整一下

#这里需要调整一下宏观变量的列名，不然到时不好按日期把dataframe进行合并
#调整一下国债与企业债数据的列名
#下面分别表示日期，AA企业债10年利率，AA+企业债10年利率，国债10年利率，国债6个月利率
marco_df[0].columns=['日期','AA_10y','AA+_10y','trea_10y','trea_6m']
#调整上证收益率和波动率的列名
marco_df[1].columns=['日期','SH_retu','SH_vol']
#调整上海银行同业拆借利率的列名
marco_df[2].columns=['日期','shibor']
#调整房地产数据的列名
marco_df[3].columns=['日期','house']
#调整无风险利率数据的列名
marco_df[4].columns=['日期','trea_1y']

## 2.3 资产负债表数据
读取资产负债表的数据，资产负债表数据的格式和前面的数据格式稍有不同，在Wind导出总负债的同时还导出了很多负债表其他的数据，可以打开excel表来看看，Acharya原文用的资产负债表的信息是一年披露一次的，然后再线性插值，但是我发现Wind的资产负债表信息披露大部分是每季都披露的，所以为了提高准确性，我最后采用的是每季披露频率更高的数据，再用线性插值的方法得到周数据

此外，但是资产负债表中可能是由于披露要求的原因，前几年银行的披露频率为年或半年，后面才开始正常每季都披露，比如北京银行2007-2008为一年披露一次，2008-2009年为每半年披露一次，到了2009以后才开始每季度披露一次，这个是在处理数据中需要注意的

**资产负债表中的数据单位都是万元**

In [147]:
bank_blance_sheet = []

for i in range(bank_research_len):
    #excel的名称大概是这样的 北京银行[601169.SH]-资产负债表.xlsx
    #然后'blance_sheet_data/'是文件的路径名
    path_name = 'blance_sheet_data/' + bank14_name[i] + '['+ bank14_code[i] + ']-资产负债表.xlsx'
    excel_read = pd.read_excel(path_name,index_col=0)
    bank_blance_sheet.append(excel_read)

下面要对读取的资产负债表信息进行整理，原因一是由于资产负债表里面有用的信息需要用到的信息只有负债总额，原因二是由于资产负债表的之前的数据是反过来的，即日期为每一列的列名，所以需要进行一个转置的操作

In [148]:
#提前资产负债表里面的总负债数据
bank_liability = []
for i in range(bank_research_len):
    temp = bank_blance_sheet[i]
    temp = temp.dropna(axis = 0,how = 'all')
    temp_T = pd.DataFrame(temp.values.T,index = temp.columns, columns = temp.index)
    temp_T.insert(0,'日期',temp_T.index)
    liability = temp_T[['日期','负债合计']]
    bank_liability.append(liability)

## 2.4 分红数据
读取分红数据，分红数据来源以中国银行为例，右下角搜中国银行，按F9，然后在分红派息的'上市以来分红统计'可以找到。  
分红的数据下载下来的时候有一张纸线图，不过还好，pd读excel的时候不会读这个折线图进来，前面几行是无效数据。  
重点要读到excel中从19行开始的历年分红数据,并且2021-03-31这个数据不用读进来，因为2021还没结束，这个数据是缺失的。  
有一些年份是没有分红的，然后里面Wind导出的没有分红年份是用--来表示的，具体可以打开excel来看看数据格式。

**分红总额的数据的单位是万元**

In [149]:
#读取分红数据
#path_name = 'dividend_data/' + '中国银行[601988.SH] - 上市以来分红统计.xlsx'
#divident = pd.read_excel(path_name)

bank_divi_raw = []
for i in range(bank_research_len):
    #文件路径例子：dividend_data/中国银行[601988.SH] - 上市以来分红统计.xlsx
    path_name = 'dividend_data/' + bank14_name[i] + '['+ bank14_code[i] + '] - 上市以来分红统计.xlsx'
    divi_file = pd.read_excel(path_name)
    bank_divi_raw.append(divi_file)

#整理一下分红数据读出来的文件,提取出有用的信息
#为了可读性，所以没把下面这个for循环和上面合并在一起
bank_divident = []
for i in range(bank_research_len):
    #因为读取的时候，可能是文件的问题，会有后面会出现很多行的NaN，所以要把全部为NaN的行给删掉
    bank_divi_raw[i] = bank_divi_raw[i].dropna(axis = 0,how = 'all') 
    #把有用的信息给截取出来，10是我一行一行数发现到这个地方刚好是有用的数据
    temp = bank_divi_raw[i].iloc[10:-1,:]
    #第一列的列名一般为‘平安银行[000001.SZ] - 上市以来分红统计’，然后这一列是记录日期的
    col_1=bank14_name[i] + '['+ bank14_code[i] + '] - 上市以来分红统计'
    #提取出日期和每年分红总额的那两列
    temp = temp[[col_1,'Unnamed: 2']]
    temp.columns = ['日期','divi_year_sum']
    bank_divident.append(temp)

因为分红总额里面，有些没有分红的年份用'--'表示了，即表示没有进行分红，为了后续计算的方便，用0进行替换

In [150]:
for i in range(bank_research_len):
    bank_divident[i] = bank_divident[i].replace('--',0)

此外比较麻烦的是，excel读进来的数据带有逗号，比如1,761,127.32，类型变成了str类型，需要处理转成float类型

In [151]:
for i in range(bank_research_len):
    len_divi=bank_divident[i].shape[0]
    for k in range(len_divi):
        #excel数据里面还混了一些int数据，int没有replace函数，所以需要先判断一些
        if type(bank_divident[i]['divi_year_sum'].iloc[k]) is str:
            bank_divident[i]['divi_year_sum'].iloc[k]=float(bank_divident[i]['divi_year_sum'].iloc[k].replace(',',''))
            #print(i)
            #bank_divident[i]['divi_year_sum'].iloc[k]=float(bank_divident[i]['divi_year_sum'].iloc[k].replace(',',''))
        
        '''print(bank_divident[i]['divi_year_sum'].iloc[k])
        if type(bank_divident[i]['divi_year_sum'].iloc[k]) is 'str':
            bank_divident[i]['divi_year_sum'].iloc[k]=float(bank_divident[i]['divi_year_sum'].iloc[k].replace(',',''))
            print(bank_divident[i]['divi_year_sum'].iloc[k])'''

#print(float(bank_divident[1]['divi_year_sum'].iloc[1].replace(',','')))

# 3 线性插值

现在要把数据求解DD的数据按日期统一在一张表当中了  
需要做以下处理  
1.把资产负债表中总负债数据进行线性插值处理  
2.调整各表的日期进行统一合并，为后续计算MertonDD做准备

先定义一个处理日期的函数（这个函数可以得到日期年与周）

In [152]:
def getweek(data):
    year_week=[]#,b='-';
    year=[];week=[]
    for i in data:
        if type(i) is float:
            i=int(i)
        i=str(i)
        if i[4] is '-': #这个是用来处理2009-08-03类的
            year_record=i[0:4];mon_record=i[5:7];day_record=i[8:10];
            date_temp=year_record+mon_record+day_record;
            #print(date_temp)
            week_record=datetime.strptime(date_temp,"%Y%m%d").strftime("%W")
            year_week_temp=year_record+week_record
            year_week_temp=int(year_week_temp); year_record = int(year_record); week_record = int(week_record)
            year_week.append(year_week_temp); year.append(year_record); week.append(week_record)
        else:
            year_record=i[0:4]; #这个是用来处理20090803类的
            date_temp=i[0:8]
            week_record=datetime.strptime(date_temp,"%Y%m%d").strftime("%W")
            #week_record=datetime.strptime(date_temp,"%Y%m%d").strftime("%W")
            year_week_temp=year_record+week_record
            year_week_temp=int(year_week_temp); year_record = int(year_record); week_record = int(week_record)
            year_week.append(year_week_temp); year.append(year_record); week.append(week_record)
    return year,week

## 3.1 线性插值编写

下面的函数linear_interpolation用于线性插值

In [153]:
#大部分年都是52周，有些年有53周，需要知道每年有多少周才能进行正确地插值
find_annual_week=[];
for i in range(1990,2024):
    #记录每年最后一天就可以知道有多少周了
    find_annual_week.append(str(i)+'1231')
#记录每年对应有多少周
year,total_week=getweek(find_annual_week)
#用字典建立每年有多少个周的索引
annual_week_record=dict(zip(year,total_week))

In [154]:
def linear_interpolation(data):
    #从起始周到结束周之间生成一个完整的周列表
    year_complete=[]; week_complete=[]
    begin_year=data['year'].iloc[0]; begin_week=data['week'].iloc[0]; #开始日期的年与周
    end_year=data['year'].iloc[-1]; end_week=data['week'].iloc[-1] #结束日期的年与周
    for i in range(begin_year,end_year+1):
        #处理第一年情况的，起始的周可能是从12周开始，并不是从第1周开始
        if i==begin_year:
            for k in range(begin_week,annual_week_record[i]+1):
                year_complete.append(i); week_complete.append(k)
        #最后一年的情况，起始是从第1周开始，但是结束并不是52或53周，可能24周这样
        elif i==end_year:
            for k in range(1,end_week):
                year_complete.append(i); week_complete.append(k)
        #中间的年份
        else:
            for k in range(1,annual_week_record[i]+1):
                year_complete.append(i); week_complete.append(k)
    year_week_complete=pd.DataFrame({'year':year_complete,'week':week_complete})
    #将原来部分数据插入到完整的年周数据当中去
    merge=pd.merge(year_week_complete,data,how='outer',on=['year','week'])
    #print(merge) #如果不清楚数据结构可以print(merge)试一下
    #下面采用线性插值的函数
    #虽然可能看到下面这行代码觉得很奇怪，用np.nan来fill nan，我debug发现interpolate函数只能处理np的NaN。。。
    merge=merge.fillna(np.nan)
    merge=merge.interpolate() #线性插值函数
    return merge

对资产负债表中的负债合计（总负债）进行线性插值处理，前面几年可能资产负债表是每年/每半年披露一次的，后面的资产负债表信息基本都是每半年披露一次的。  
现在要做的就是通过线性插值方法把数据全部转换以周为频率的数据

In [155]:
#先获得日期的年和周的数据
for i in range(bank_research_len):
    year,week=getweek(bank_liability[i]['日期'])
    bank_liability[i].insert(0,'year',year)
    bank_liability[i].insert(1,'week',week)
    #按日期从小到大进行排序
    bank_liability[i]=bank_liability[i].sort_values(by=['year','week'])
    #进行插值只需要年，周和负债合计这三列，所以只提取这三列
    bank_liability[i]=bank_liability[i][['year','week','负债合计']]
    bank_liability[i]=bank_liability[i].rename(columns={'负债合计':'liability'})
    
liability_interpolate=[]#记录每个银行线性插值后的总负债

#进行线性插值
for i in range(bank_research_len):
    get_liability=linear_interpolation(bank_liability[i])
    liability_interpolate.append(get_liability)

## 3.2 合并MertonDD需要的数据
后续把所有计算每个银行MertonDD的数据合并在一张表中

In [194]:
#银行总市值数据V_E
market_value=[]
for i in range(bank_research_len):
    year,week=getweek(bank14_market_value[i]['日期'])
    bank14_market_value[i].insert(0,'year',year); bank14_market_value[i].insert(0,'week',week)
    #只需要year,week和market_value这三列的数据
    bank14_market_value[i]=bank14_market_value[i][['year','week','market_value']]
    market_value.append(bank14_market_value[i])

market_value_data=market_value
#设置DD需要的总表的数据，后面不断加入对应数据
#DD_all_data=market_value

这里还需要计算银行总市值即(market value of equity)的标准差$S_E$，文中指出的是用过去12个月（约为52周）的标准差来处理  
不过实际处理中前面是有些时间段是不满12个月，于是我用了前面可以利用的所有能够利用的数据（最大信息）为来处理

In [157]:
def get_SE(data):
    return np.std(data)

#求每个银行每个时间段的SE
SE=[]
for i in range(bank_research_len):
    observation_len=market_value[i].shape[0]
    single_bank_SE=[]
    for k in range(1,observation_len+1):
        if k < 52:
            #计算前k个market_value的标准差
            SE_temp=get_SE(market_value[i]['market_value'].iloc[:k])
            single_bank_SE.append(SE_temp)
        else:
            #过去12个月（即52周）的标准差
            SE_temp=get_SE(market_value[i]['market_value'].iloc[k-52:k])
            single_bank_SE.append(SE_temp)
    SE.append(single_bank_SE)

#将SE的数据插入到DD总的数据DD_all_data当中去
for i in range(bank_research_len):
    DD_all_data[i].insert(3,'std_m_v',SE[i])

分红的数据（前一年的分红总额）也要加入表中，后续要计算DD

In [158]:
for i in range(bank_research_len):
    year,week=getweek(bank_divident[i]['日期'])
    bank_divident[i].insert(0,'year',year)
    #只需要年和分红总额这两列
    year_divident=bank_divident[i][['year','divi_year_sum']]
    DD_all_data[i]=pd.merge(DD_all_data[i],year_divident,how='inner',on='year')

负债的数据合并到表中

In [159]:
for i in range(bank_research_len):
    DD_all_data[i]=pd.merge(DD_all_data[i],liability_interpolate[i],how='inner',on=['year','week'])

加入无风险利率的数据，不过比较可惜的是从Wind导出的无风险利率的数据是只有从2007年开始的，所以资产负债表的前面有3-4年的样本数据被截断掉了，比如2002-2006年就被舍弃掉了

In [160]:
#调整无风险利率的格式
year,week=getweek(marco_df[4]['日期'])
marco_df[4].insert(0,'year',year)
marco_df[4].insert(1,'week',week)
free_risk=marco_df[4][['year','week','trea_1y']]

for i in range(bank_research_len):
    DD_all_data[i]=pd.merge(DD_all_data[i],free_risk,how='inner',on=['year','week'])

In [161]:
#DD需要的所有数据格式大概就是这样的，可以print下面出来看看
print(DD_all_data[1])

     year  week  market_value       std_m_v divi_year_sum     liability  \
0    2007    51  2.139554e+07  5.490855e+06       69678.1  8.767289e+07   
1    2007    52  2.299378e+07  5.442568e+06       69678.1  8.817057e+07   
2    2008     1  2.682172e+07  5.450950e+06        130211  8.891387e+07   
3    2008     2  2.484461e+07  5.439063e+06        130211  8.915949e+07   
4    2008     3  2.316798e+07  5.394302e+06        130211  8.940511e+07   
5    2008     4  2.056811e+07  5.288711e+06        130211  8.965073e+07   
6    2008     5  2.238410e+07  5.252575e+06        130211  8.989635e+07   
7    2008     6  2.126054e+07  5.149902e+06        130211  9.014197e+07   
8    2008     7  1.741082e+07  5.015539e+06        130211  9.038758e+07   
9    2008     8  1.834712e+07  4.911432e+06        130211  9.063320e+07   
10   2008     9  1.717566e+07  4.753591e+06        130211  9.087882e+07   
11   2008    10  1.412724e+07  4.669454e+06        130211  9.112444e+07   
12   2008    11  1.492854

# 4 计算Mertondd

## 4.1 Distance to default(DD)的计算原理
DD本身的计算类似于一个call-option，所以形式和期权的形式是十分类似的
计算的关键在于解一个非线性方程组，这个非线性方程组由下面这个等式构成，$V_A$和$s_A$是未知的，需要求解出来用于后续DD计算

$$
V_{E}=V_{A} e^{-d T} N\left(d_{1}\right)-X e^{-r T} N\left(d_{2}\right)+\left(1-e^{-d T}\right) V_{A} \tag1
$$
$$
s_{E}=\frac{V_{A} e^{-d T} N\left(d_{1}\right) s_{A}}{V_{E}} \tag2
$$


在上述公式中，$V_E$是银行的市值（即market value of equity)，$X$是银行资产负债表中的总债务，$r$是无风险利率，$s_E$是银行权益波动的标准差，N为正态分布cdf的对应值，根据Acharya提出的线性插值方法可以得到权益波动的标准差，这些都是可以通过已知数据得到。
并且$d_1$和$d_2$由下面式子给出
$$
d_{1}=\frac{\log \left(\frac{V_{A}}{X}\right)+\left(r-d+\frac{s_{A}^{2}}{2}\right) T}{s_{A} \sqrt{T}} \tag3
$$
$$
d_{2}=d_{1}-s_{A} \sqrt{T} \tag4
$$
另外$d$代表是分红率，文中定义为前一年的分红总额除以资产的市场价值$V_A$,即$d=\frac{divident\_last\_year}{V_A}$  
我们需要求解(1)(2)(3)(4)这4个非线性方程组，然后得到$V_A$和$s_A$
再把这两个值代入到Mertondd的计算公式里面，就可以得到DD了

$$
\text { Mertondd }=\frac{\log \left(\frac{V_{A}}{X}\right)+\left(m-d-\frac{s_{A}^{2}}{2}\right) T}{s_{A} \sqrt{T}}
$$

## 4.2 求解DD中的非线性方程组

原文中Acharya用的数据是每年的资产负债表数据，但是我发现中国银行资产负债表的数据基本是每个季度披露一次的，为了更加准确，所以我用的是季度数据，因此T=1年，变成了T=0.25年，无风险利率$r$和分红率$d$都要除4，这个地方稍有些不同  
文中建议$V_A$和$s_A$的初始迭代值为$V_A=V_E+X$以及$s_A=s_E \times V_E/(V_E+X)$

这里给了一个求解上述非线性方程组的例子，用DD_all_data[0]的第一行当例子，np.array里面4个等式分别对应上面公式的(1)(2)(3)(4)

In [168]:
row1=DD_all_data[0].iloc[1,:]
#x[0]为V_A，x[1]为s_A，x[2]=d1,x[3]=d2
V_E=row1[2]; s_E=row1[3]; div_last=row1[4]/4; r=row1[6] /4 *0.01; X=row1[5]; T=0.25#r*0.01是因为要将3.7%变成0.037

def DD_func(x):
    #把等式左边移到右边去，即等式左边加个-号，4个式子，一个对应一行
    return np.array([-V_E+x[0]*np.exp(-div_last /x[0]*T)*stats.norm.cdf(x[2])-X*np.exp(-r*T)*stats.norm.cdf(x[3])+(1-np.exp(-div_last/x[0]*T))*x[0],
                    -s_E+x[0]*np.exp(-div_last/x[0]*T)*stats.norm.cdf(x[2])*x[1]/V_E,
                    -x[2]+(np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T)),
                    -x[3]+x[2]-x[1]*np.sqrt(T)])

sol3_fsolve = fsolve(DD_func,[V_E+X,s_E*V_E/(V_E+X),0,0])


但是上面的话，需要给d1和d2的初始迭代值，我上面设成了0和0，但Acharya原文只给了$V_A$和$s_A$的初始迭代值为$V_A=V_E+X$以及$s_A=s_E/(V_E+X)$  
在上面的基础上，我们需要将d1(对应 x[2] )和d2(对应 x[3] )代入到（1）和（2）中，所以下面形式也比较复杂  
下面这个仍是以DD_all_data[0]的第一行为例子

In [170]:
row1=DD_all_data[0].iloc[1,:]
#x[0]为V_A，x[1]为s_A，x[2]=d1,x[3]=d2
V_E=row1[2]; s_E=row1[3]; div_last=row1[4]/4; r=row1[6] /4 *0.01; X=row1[5]; T=0.25

def DD_fun(x):
    return np.array([-V_E+x[0]*np.exp(-div_last /x[0]*T)*stats.norm.cdf((np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T)))-X*np.exp(-r*T)*stats.norm.cdf((np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T))-x[1]*np.sqrt(T))+(1-np.exp(-div_last/x[0]*T))*x[0],
                    -s_E+x[0]*np.exp(-div_last/x[0]*T)*stats.norm.cdf((np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T)))*x[1]/V_E])

sol3_fsolve=fsolve(DD_fun,[V_E+X,s_E*V_E/(V_E+X)])


下面正式的开始批量的计算$V_A$和$s_A$ (这个解非线性方程组还是蛮快的，大约30s能跑完）

In [172]:
V_A_all=[]; s_A_all=[]
for i in range(bank_research_len):
    len_observation=DD_all_data[i].shape[0]
    #len_observation=2 #可以用2个的先试一下
    V_A_single=[]; s_A_single=[]
    for k in range(len_observation):
        row=DD_all_data[i].iloc[k,:]
        V_E=row[2]; s_E=row[3]; div_last=row[4]/4; r=row[6]/4*0.01; X=row[5]; T=0.25
        def DD_fun(x):
            return np.array([-V_E+x[0]*np.exp(-div_last /x[0]*T)*stats.norm.cdf((np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T)))-X*np.exp(-r*T)*stats.norm.cdf((np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T))-x[1]*np.sqrt(T))+(1-np.exp(-div_last/x[0]*T))*x[0],
                            -s_E+x[0]*np.exp(-div_last/x[0]*T)*stats.norm.cdf((np.log(x[0]/X)+(r-div_last/x[0]+(x[1]**2)/2)*T)/(x[1]*np.sqrt(T)))*x[1]/V_E])
        solve_res=fsolve(DD_fun,[V_E+X,s_E*V_E/(V_E+X)])
        V_A_single.append(solve_res[0]); s_A_single.append(solve_res[1])
    V_A_all.append(V_A_single); s_A_all.append(s_A_single)

把求出来的$V_A$和$s_A$插入到DD_all_data当中，我检查了一遍，发现其实算出来的$V_A$和$V_E$以及$s_A$和$s_E$还是蛮接近的

In [173]:
for i in range(bank_research_len):
    DD_all_data[i].insert(7,'V_A',V_A_all[i])
    DD_all_data[i].insert(8,'s_A',s_A_all[i])

## 4.3 代入数据计算DD

$$
\text { Mertondd }=\frac{\log \left(\frac{V_{A}}{X}\right)+\left(m-d-\frac{s_{A}^{2}}{2}\right) T}{s_{A} \sqrt{T}}
$$
具体公式如上，式子中的变量我们都计算得到了，然后Acharya指出m根据参考文献取为6%，因此开始计算

In [176]:
DD_values=[]
for i in range(bank_research_len):
    len_obs=DD_all_data[i].shape[0]
    Merton_DD_single=[]
    for k in range(len_obs):
        row=DD_all_data[i].iloc[k,:]
        V_A=row[7]; s_A=row[8]; T=0.25; m=0.06/4; X=row[5]; d=row[4]/V_A/4 #分红率为分红总额/资产总值/4
        Merton_DD=(np.log(V_A/X) + (m-d-(s_A**2)/2)*T)/(s_A*np.sqrt(T))
        Merton_DD_single.append(Merton_DD)
    DD_values.append(Merton_DD_single)

已经计算好MertonDD的值了，将这个插入总表里面进去

In [182]:
for i in range(bank_research_len):
    DD_all_data[i].insert(9,'DD_values',DD_values[i])

# 5 计算$\Delta \text { CovarSystem }_{t}^{q}$

$$
\Delta \operatorname{Bank} D D_{i, t}=\propto_{i}+\gamma_{i} M_{t-1}+\varepsilon_{i, t}
$$
$$
\Delta \text { SystemDD }_{t}=\propto_{\text {system } \mid i}+\beta_{\text {system } \mid i} \Delta \text { BankDD }_{i, t}+\gamma_{\text {system } \mid i} M_{t-1}+\varepsilon_{\text {system } \mid i, t}
$$
其中$\Delta \operatorname{Bank} D D_{i, t}$是银行$i$在$t$时期的变化值，即$t$期的值减去$t-1$期的值。  
而$\Delta \text { SystemDD }_{t}$是$t$时期所有样本中的银行以市值为权重平均得到的$BankDD_t$的值，减去$t-1$加权平均得到的$BankDD_{t-1}$得到的值

$$
\Delta \text { CovarSystem }_{t}^{q}=\hat{\beta}_{\text {system } \mid i}^{q}\left(\Delta \widehat{B a n k DD} _{i, t}^{q}-\Delta \widehat{BankDD} _{i, t}^{50 \%}\right)
$$

## 5.1 计算$\Delta$BankDD和$\Delta$SystemDD

计算$\Delta BankDD$

In [185]:
#定义一个差分的函数，即t期减去t-1期，因为后面经常要对时间差分
def diff1(data):
    data_diff=data-data.shift(1)
    return data_diff

In [187]:
for i in range(bank_research_len):
    DD_all_data[i]['change_DD']=diff1(DD_all_data[i]['DD_values'])

计算$\Delta SystemDD$

In [200]:
#先要求出金融系统总市值才能得到加权平均的权重
sys_mar_val=market_value_data[0]
for i in range(1,bank_research_len):
    sys_mar_val=pd.merge(sys_mar_val,market_value_data[i],how='outer',on=['year','week'])

sys_mar_val.fillna(0,inplace=True)
sys_mar_val['num']=sys_mar_val.iloc[:,2:bank_research_len+1].sum(axis=1)

